**BRING TOGETHER FEATURES ENGINEERED FROM POSTCODES TO CONSTRUCT MAIN DATAFRAME FOR MODELLING**

The data from the Land Registry provides us with postcodes for over three hundred thousand transactions.

We can now use those postcodes to engineer features for each of these properties, based on the data that has been sourced for economic factors, transport, schools, crime, and centrality.

The new dataframe will then be used for predictive modelling.


In [1]:
import pandas as pd
import geopandas as gpd
import joblib
import matplotlib.pyplot as plt

In [2]:
# start building the dataframe with the cleaned up transaction data from the Land Registry, note duplicates have already been dropped

df = pd.read_csv('../big_files/cleaned_transactions.csv')

In [3]:
# import list of all London postcodes with their geo-coordinates to merge with main dataframe

london_postcodes = pd.read_csv('../big_files/london_postcodes.csv')
london_postcodes = london_postcodes.drop(columns = ['Unnamed: 0'])

In [4]:
# merge the transaction dataframe with the geocoordinates from the postcodes
# transform into geo-dataframe and set CRS

df = pd.merge(df, london_postcodes, how = 'inner', left_on = 'postcode', right_on = 'postcode')
df = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['longitude'], df['latitude']))
df = df.set_crs(epsg = 4326)
df = df.to_crs(epsg = 27700)

In [5]:
# import shape file for all LSOA boundaries and set CRS

lsoa = gpd.read_file('../big_files/LSOA_2011_London_gen_MHW.shp')
lsoa = lsoa[['LSOA11CD', 'geometry']]
lsoa = lsoa.set_crs(epsg = 27700)

In [6]:
# spatial join to merge LSOA shape file with transaction_data

df = gpd.sjoin(df, lsoa, how = 'left', op = 'intersects')
df = df.rename(columns = {'LSOA11CD': 'lsoa'})
df = df.dropna()
df = df.drop(columns = ['index_right'])

In [7]:
# import free school meal data and merge it with transaction dataframe

school_meals = pd.read_csv('../big_files/school_meals.csv')
school_meals = school_meals.rename(columns = {'pupil_perc_fsm_lsoa': 'fsm_lsoa'})
school_meals = school_meals.drop(columns = ['Unnamed: 0'])
df = pd.merge(df, school_meals, how = 'left', left_on = 'lsoa', right_on = 'lsoa')

In [8]:
# import ward data to link to lsoa in transaction dataframe

ward_lsoa = pd.read_csv('../big_files/ward_lsoa.csv')
ward_lsoa = ward_lsoa.drop(columns = ['Unnamed: 0'])
df = pd.merge(df, ward_lsoa, how = 'left', left_on = 'lsoa', right_on = 'lsoa')

In [9]:
# import crime survey data, set CRS, merge with transaction dataframe

bcp = gpd.read_file('../big_files/borough_crime_perception_gdf.shp')
bcp = bcp.drop(columns = ['GSS_CODE', 'HECTARES', 'NONLD_AREA', 'ONS_INNER', 'SUB_2009', 'SUB_2006'])
bcp = bcp.set_crs(epsg = 27700)
bcp = bcp.drop(columns = ['geometry'])
df = pd.merge(df, bcp, how = 'left', left_on = 'borough', right_on = 'borough_na')

In [10]:
#data missing from City of London area, fill with average for all boroughs

df['Are gangs'] = df['Are gangs'].fillna(df['Are gangs'].mean())
df['Are you wo'] = df['Are you wo'].fillna(df['Are you wo'].mean())
df['How safe d'] = df['How safe d'].fillna(df['How safe d'].mean())
df['Are you sa'] = df['Are you sa'].fillna(df['Are you sa'].mean())
df = df.drop(columns = ['borough_na'])
df = df.reset_index(drop = True)

In [11]:
# import centrality data, adjust to match length of dataframe, set CRS, add to dataframe

trafalgar_square = gpd.read_file('../big_files/trafalgar_square_gdf.shp')
trafalgar_square = trafalgar_square.reset_index(drop = True)
trafalgar_square = trafalgar_square.drop(columns = ['FID'])
trafalgar_square_1 = trafalgar_square.loc[:len(df) -1]
trafalgar_square_1 = trafalgar_square_1.set_crs(epsg = 27700)
trafalgar_square_1 = trafalgar_square_1.to_crs(epsg = 27700)
df['dist_traf_square'] = df.distance(trafalgar_square_1)

In [12]:
# import crime data grouped by LSOA, merge with final dataframe, fill those few LSOAs where there was no recorded crime

grouped_crime = pd.read_csv('../big_files/grouped_crime.csv')
grouped_crime = grouped_crime.drop(columns = ['Unnamed: 0'])
df = pd.merge(df, grouped_crime, how = 'left', left_on = 'lsoa', right_on = 'lsoa')
df['crime_count_lsoa'] = df['crime_count_lsoa'].fillna(0)


In [14]:
# import deprivation data and merge with final dataframe

deprivation = pd.read_csv('../big_files/deprivation_clean.csv')
deprivation = deprivation.drop(columns = ['Unnamed: 0'])
df = pd.merge(df, deprivation, how = 'left', left_on = 'lsoa', right_on = 'lsoa')

In [15]:
# import gun crime data and merge with final dataframe, fill those wards that didn't record any gun crime

gun = pd.read_csv('../big_files/gun.csv')
gun = gun.loc[:, ['ward_code', 'total_gun_crimes']]
df = pd.merge(df, gun, how = 'left', left_on = 'ward_code', right_on = 'ward_code')
df['total_gun_crimes'] = df['total_gun_crimes'].fillna(0)

In [17]:
# import knife crime data and merge with final dataframe, fill missing values (City of London)

knife = pd.read_csv('../big_files/knife.csv')
knife = knife.loc[:, ['ward_code', 'total_knife_crimes']]
df = pd.merge(df, knife, how = 'left', left_on = 'ward_code', right_on = 'ward_code')
df['total_knife_crimes'] = df['total_knife_crimes'].fillna(df['total_knife_crimes'].mean())

In [19]:
# merge ward and lsoa data, ready to aggregate data on estate agents, delicatessens etc

lsoa = lsoa.rename(columns = {'LSOA11CD': 'lsoa'})
ward_lsoa = pd.merge(ward_lsoa, lsoa, how = 'left', left_on = 'lsoa', right_on = 'lsoa')
ward_lsoa = gpd.GeoDataFrame(ward_lsoa, geometry = ward_lsoa['geometry'])
ward_lsoa = ward_lsoa.set_crs(epsg = 27700)
ward_lsoa = ward_lsoa.to_crs(epsg = 27700)

In [20]:
# import estate agent data, set CRS, merge with ward/lsoa data, groupby wards to find total/ward, merge with transaction dataframe
# some wards don't have any estate agents

estate_agents = gpd.read_file('../big_files/estate_agents_gdf.shp')
estate_agents = estate_agents.set_crs(epsg = 4326)
estate_agents = estate_agents.to_crs(epsg = 27700)
estate_agents_ward = gpd.sjoin(estate_agents, ward_lsoa, how = 'left', op = 'intersects')
ea_wards = estate_agents_ward.groupby('ward_code')['postcode'].count().reset_index()
ea_wards = ea_wards.rename(columns = {'postcode': 'ea_count'})
df = pd.merge(df, ea_wards, how = 'left', left_on = 'ward_code', right_on = 'ward_code')
df['ea_count'] = df['ea_count'].fillna(0)

In [21]:
# import delicatessen data, merge with ward/lsoa data, groupby wards to find total/ward, merge with transaction df
# some wards don't have any delicatessens

delicatessens_gdf = gpd.read_file('../big_files/delicatessens_gdf.shp')
delicatessens_gdf = delicatessens_gdf.drop(columns = ['name', 'Unnamed_ 0', 'latitude', 'longitude'])
delicatessen_wards = ward_lsoa.drop(columns = ['lsoa', 'ward_name', 'borough'])
delicatessen_wards = delicatessen_wards.drop_duplicates()
delicatessen_wards = gpd.sjoin(delicatessen_wards, delicatessens_gdf, how = 'left', op = 'intersects')
delicatessen_wards = delicatessen_wards.drop_duplicates(subset = ['ward_code'])
grouped_delis = delicatessen_wards.groupby('ward_code')['postcode'].count().reset_index()
grouped_delis = grouped_delis.rename(columns = {'postcode': 'deli_count'})
df = pd.merge(df, grouped_delis, how = 'left', left_on = 'ward_code', right_on = 'ward_code')

In [24]:
# import florist data, merge with ward/lsoa data, groupby wards to find total/ward, merge with transaction df
# some wards don't have any florists

florists = gpd.read_file('../big_files/florists.shp')
florists_wards = ward_lsoa.drop(columns = ['lsoa', 'ward_name', 'borough'])
florists_wards = gpd.sjoin(florists_wards, florists, how = 'left', op = 'intersects')
florists_wards = florists_wards.drop_duplicates(subset = ['ward_code'])
grouped_florists = florists_wards.groupby('ward_code')['postcode'].count().reset_index()
grouped_florists = grouped_florists.rename(columns = {'postcode': 'florist_count'})
df = pd.merge(df, grouped_florists, how = 'left', left_on = 'ward_code', right_on = 'ward_code')

In [25]:
# import restaurant data, merge with ward/lsoa data, groupby wards to find total/ward, merge with transaction df
# some wards don't have any restaurants

restaurants_gdf = gpd.read_file('../big_files/restaurants_gdf.shp')
restaurants_wards = ward_lsoa.drop(columns = ['lsoa', 'ward_name', 'borough'])
restaurants_wards = restaurants_wards.drop_duplicates(subset = ['ward_code'])
restaurants_wards = gpd.sjoin(restaurants_wards, restaurants_gdf, how = 'left', op = 'intersects')
grouped_restaurants = restaurants_wards.groupby('ward_code')['postcode'].count().reset_index()
grouped_restaurants = grouped_restaurants.rename(columns = {'postcode': 'restaurant_count'})
df = pd.merge(df, grouped_restaurants, how = 'left', left_on = 'ward_code', right_on = 'ward_code')

In [28]:
# import location data for 'outstanding' schools, including 500m radius around them
# separate main dataframe into properties that fall inside these areas, and those that don't

schools_500m = gpd.read_file('../big_files/schools_500m.shp')
inside_500m = gpd.sjoin(df, schools_500m, how = 'inner', op = 'intersects')
inside_500m = inside_500m.drop_duplicates(subset = ['postcode'])
ind_500 = list(inside_500m.index)
outside_500m = df.drop(ind_500)
outside_500m['school_pos'] = 'None'
df = pd.concat([inside_500m, outside_500m])
df = df.drop(columns = ['index_right'])

In [30]:
# import location data for tube stops, including circles of various sizes around them

tubes_2400m = gpd.read_file('../big_files/tubes2400m.shp')
tubes_1600m = gpd.read_file('../big_files/tubes1600m.shp')
tubes_800m = gpd.read_file('../big_files/tubes800m.shp')
tubes_400m = gpd.read_file('../big_files/tubes400m.shp')
tubes_200m = gpd.read_file('../big_files/tubes200m.shp')
tubes_50m = gpd.read_file('../big_files/tubes50m.shp')

In [31]:
#split dataframe into those properties within 50m of tube, and those that aren't

inside_50m = gpd.sjoin(df, tubes_50m, how = 'inner', op = 'intersects')
ind_50 = list(inside_50m.index)
outside_50m = df.drop(ind_50)

In [32]:
# create segment of main dataframe between 50m and 200m from nearest tube
#dropping duplicates as some properties are getting counted twice if they are close to two tube stops

between50_200 = gpd.sjoin(outside_50m, tubes_200m, how = 'inner', op = 'intersects')
between50_200 = between50_200.drop_duplicates(subset = ['postcode'])

In [33]:
#split dataframe into those properties within 200m of tube, and those that aren't

ind_200 = list(between50_200.index)
outside_200m = outside_50m.drop(ind_200)

In [35]:
# create segment of main dataframe between 200m and 400m from nearest tube
#dropping duplicates as some properties are getting counted twice if they are close to two tube stops

between200_400 = gpd.sjoin(outside_200m, tubes_400m, how = 'inner', op = 'intersects')
between200_400 = between200_400.drop_duplicates(subset = ['postcode'])

In [36]:
#split dataframe into those properties within 400m of tube, and those that aren't

ind_400 = list(between200_400.index)
outside_400m = outside_200m.drop(ind_400)

In [37]:
# create segment of main dataframe between 400m and 800m from nearest tube
#dropping duplicates as some properties are getting counted twice if they are close to two tube stops

between400_800 = gpd.sjoin(outside_400m, tubes_800m, how = 'inner', op = 'intersects')
between400_800 = between400_800.drop_duplicates(subset = ['postcode'])

In [38]:
#split dataframe into those properties within 800m of tube, and those that aren't

ind_800 = list(between400_800.index)
outside_800m = outside_400m.drop(ind_800)

In [39]:
# create segment of main dataframe between 800m and 1600m from nearest tube
#dropping duplicates as some properties are getting counted twice if they are close to two tube stops

between800_1600 = gpd.sjoin(outside_800m, tubes_1600m, how = 'inner', op = 'intersects')
between800_1600 = between800_1600.drop_duplicates(subset = ['postcode'])

In [40]:
#split dataframe into those properties within 1600m of tube, and those that aren't

ind_1600 = list(between800_1600.index)
outside_1600m = outside_800m.drop(ind_1600)

In [41]:
# create segment of main dataframe between 1600m and 2400m from nearest tube
#dropping duplicates as some properties are getting counted twice if they are close to two tube stops

between1600_2400 = gpd.sjoin(outside_1600m, tubes_2400m, how = 'inner', op = 'intersects')
between1600_2400 = between1600_2400.drop_duplicates(subset = ['postcode'])

In [42]:
#split dataframe into those properties within 2400m of tube, and those that aren't

ind_2400 = list(between1600_2400.index)
outside_2400m = outside_1600m.drop(ind_2400)

In [43]:
# drop unecessary column from new dataframes

inside_50m = inside_50m.drop(columns = ['index_right'])
between50_200 = between50_200.drop(columns = ['index_right'])
between200_400 = between200_400.drop(columns = ['index_right'])
between400_800 = between400_800.drop(columns = ['index_right'])
between800_1600 = between800_1600.drop(columns = ['index_right'])
between1600_2400 = between1600_2400.drop(columns = ['index_right'])

In [44]:
# different cleaning for each of the new dataframes, can't write a single function

inside_50m['zone_50m'] = inside_50m['zone_50m'].str[0].astype(float)
inside_50m['zone_50m'] = inside_50m['zone_50m'].fillna(-1)
inside_50m['zone_200m'] = -1
inside_50m['zone_400m'] = -1
inside_50m['zone_800m'] = -1
inside_50m['zone_1600m'] = -1
inside_50m['zone_2400m'] = -1
between50_200['zone_200m'] = between50_200['zone_200m'].str[0].astype(float)
between50_200['zone_200m'] = between50_200['zone_200m'].fillna(-1)
between50_200['zone_50m'] = -1
between50_200['zone_400m'] = -1
between50_200['zone_800m'] = -1
between50_200['zone_1600m'] = -1
between50_200['zone_2400m'] = -1
between200_400['zone_400m'] = between200_400['zone_400m'].str[0].astype(float)
between200_400['zone_400m'] = between200_400['zone_400m'].fillna(-1)
between200_400['zone_50m'] = -1
between200_400['zone_200m'] = -1
between200_400['zone_800m'] = -1
between200_400['zone_1600m'] = -1
between200_400['zone_2400m'] = -1
between400_800['zone_800m'] = between400_800['zone_800m'].str[0].astype(float)
between400_800['zone_800m'] = between400_800['zone_800m'].fillna(-1)
between400_800['zone_50m'] = -1
between400_800['zone_200m'] = -1
between400_800['zone_400m'] = -1
between400_800['zone_1600m'] = -1
between400_800['zone_2400m'] = -1
between800_1600['zone_1600m'] = between800_1600['zone_1600m'].str[0].astype(float)
between800_1600['zone_1600m'] = between800_1600['zone_1600m'].fillna(-1)
between800_1600['zone_50m'] = -1
between800_1600['zone_200m'] = -1
between800_1600['zone_400m'] = -1
between800_1600['zone_800m'] = -1
between800_1600['zone_2400m'] = -1
between1600_2400['zone_2400m'] = between1600_2400['zone_2400m'].str[0].astype(float)
between1600_2400['zone_2400m'] = between1600_2400['zone_2400m'].fillna(-1)
between1600_2400['zone_50m'] = -1
between1600_2400['zone_200m'] = -1
between1600_2400['zone_400m'] = -1
between1600_2400['zone_800m'] = -1
between1600_2400['zone_1600m'] = -1
outside_2400m['zone_50m'] = -1
outside_2400m['zone_200m'] = -1
outside_2400m['zone_400m'] = -1
outside_2400m['zone_800m'] = -1
outside_2400m['zone_1600m'] = -1
outside_2400m['zone_2400m'] = -1


In [45]:
# concatenate new dataframes into one

df = pd.concat([inside_50m, between50_200, between200_400, between400_800, between800_1600, between1600_2400, outside_2400m])


In [50]:
#final cleaning for new dataframe

school = df['school_pos'] != 'None'
no_school = df['school_pos'] == 'None'
df.loc[school, 'school_pos'] = 1
df.loc[no_school, 'school_pos'] = 0

In [51]:
df['tube_zone'] = 0
zone_1 = (df['zone_50m'] == 1) | (df['zone_200m'] == 1) | (df['zone_400m'] == 1) | (df['zone_800m'] == 1) | (df['zone_1600m'] == 1) | (df['zone_2400m'] == 1)
zone_2 = (df['zone_50m'] == 2) | (df['zone_200m'] == 2) | (df['zone_400m'] == 2) | (df['zone_800m'] == 2) | (df['zone_1600m'] == 2) | (df['zone_2400m'] == 2)
zone_3 = (df['zone_50m'] == 3) | (df['zone_200m'] == 3) | (df['zone_400m'] == 3) | (df['zone_800m'] == 3) | (df['zone_1600m'] == 3) | (df['zone_2400m'] == 3)
zone_4 = (df['zone_50m'] == 4) | (df['zone_200m'] == 4) | (df['zone_400m'] == 4) | (df['zone_800m'] == 4) | (df['zone_1600m'] == 4) | (df['zone_2400m'] == 4)
zone_5 = (df['zone_50m'] == 5) | (df['zone_200m'] == 5) | (df['zone_400m'] == 5) | (df['zone_800m'] == 5) | (df['zone_1600m'] == 5) | (df['zone_2400m'] == 5)

In [52]:
df.loc[zone_1, 'tube_zone'] = 1
df.loc[zone_2, 'tube_zone'] = 2
df.loc[zone_3, 'tube_zone'] = 3
df.loc[zone_4, 'tube_zone'] = 4
df.loc[zone_5, 'tube_zone'] = 5

In [53]:
df['zone_50m'] = df['zone_50m'].replace(-1, 0)
df['zone_200m'] = df['zone_200m'].replace(-1, 0)
df['zone_400m'] = df['zone_400m'].replace(-1, 0)
df['zone_800m'] = df['zone_800m'].replace(-1, 0)
df['zone_1600m'] = df['zone_1600m'].replace(-1, 0)
df['zone_2400m'] = df['zone_2400m'].replace(-1, 0)

In [54]:
z50 = df['zone_50m'] != 0
z200 = df['zone_200m'] != 0
z400 = df['zone_400m'] != 0
z800 = df['zone_800m'] != 0
z1600 = df['zone_1600m'] != 0
z2400 = df['zone_2400m'] != 0

In [55]:
df.loc[z50,'zone_50m'] = 1
df.loc[z200,'zone_200m'] = 1
df.loc[z400,'zone_400m'] = 1
df.loc[z800,'zone_800m'] = 1
df.loc[z1600,'zone_1600m'] = 1
df.loc[z2400,'zone_2400m'] = 1

In [56]:
# introduce a final feature, 'any_tube' - is there any tube stop within 2400m of property?

df['any_tube'] = df['zone_50m'] + df['zone_200m'] + df['zone_400m'] + df['zone_800m'] + df['zone_1600m'] + df['zone_2400m']

In [ ]:
# geographic_features = ['fsm_lsoa', 'ea_in_ward', 'avg_airbnb', 'airbnb_tot', 'crime_lsoa', 'deli_count', 'flor_count', 'rest_count', 'income_rank_pos','employment_rank_pos', 'education_rank_pos', 'health_dep_score', 'crime_rank_pos', 'housing_rank_pos', 'living_env_pos', 'zone_50m','zone_200m', 'zone_400m', 'zone_800m', 'zone_1600m', 'zone_2400m', 'dist_traf', 'gang_prob_pos', 'crime_worry_pos', 'safety_fears_pos', 'satisfaction_pos', 'gun_crime', 'knife_crime', 'good_school', 'tube_zone', 'any_tube']

In [59]:
# rename columns 

df = df.rename(columns = {'school_pos': 'good_school', 'avg_airbnb_pr_lsoa':'avg_airbnb', 'airbnb_tot_lsoa': 'airbnb_tot', 'crime_count_lsoa': 'crime_lsoa', 'Are gangs': 'gang_prob_pos', 'Are you wo': 'crime_worry_pos', 'How safe d': 'safety_fears_pos', 'Are you sa': 'satisfaction_pos', 'dist_traf_square': 'dist_traf', 'crime_count_lsoa': 'crime_lsoa', 'Income Rank (where 1 is most deprived)': 'income_rank_pos', 'Employment Rank (where 1 is most deprived)': 'employment_rank_pos', 'Education, Skills and Training Rank (where 1 is most deprived)': 'education_rank_pos', 'Health Deprivation and Disability Score': 'health_dep_score', 'Crime Rank (where 1 is most deprived)': 'crime_rank_pos', 'Barriers to Housing and Services Rank (where 1 is most deprived)':'housing_rank_pos', 'Living Environment Score': 'living_env_pos', 'school_pos': 'good_school','total_gun_crimes': 'gun_crime', 'total_knife_crimes': 'knife_crime', 'ea_count': 'ea_in_ward', 'florist_count': 'flor_count', 'restaurant_count': 'rest_count'})

In [63]:
# switch to categorical dtype ahead of dummifying

df['zone_50m'] = df['zone_50m'].astype('category')
df['zone_200m'] = df['zone_200m'].astype('category')
df['zone_400m'] = df['zone_400m'].astype('category')
df['zone_800m'] = df['zone_800m'].astype('category')
df['zone_1600m'] = df['zone_1600m'].astype('category')
df['zone_2400m'] = df['zone_2400m'].astype('category')
df['good_school'] = df['good_school'].astype('category')
df['tube_zone'] = df['tube_zone'].astype('category')

In [64]:
df.head()

,price,date,postcode,property_type,new_build,estate_type,district,transaction_type,year,latitude,...,rest_count,good_school,zone_50m,zone_200m,zone_400m,zone_800m,zone_1600m,zone_2400m,tube_zone,any_tube
264450,455000,2017-03-31,E1 1BY,F,N,L,TOWER HAMLETS,A,2017,51.519554,...,0,1,1.0,0.0,0.0,0.0,0.0,0.0,2,1.0
264451,380000,2018-12-12,E1 1BY,F,N,L,TOWER HAMLETS,A,2018,51.519554,...,0,0,1.0,0.0,0.0,0.0,0.0,0.0,2,1.0
264557,230000,2017-05-26,E1 7AN,F,N,L,TOWER HAMLETS,A,2017,51.515408,...,0,1,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0
264558,425000,2017-10-27,E1 7AN,F,N,L,TOWER HAMLETS,A,2017,51.515408,...,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0
298045,720000,2018-04-16,E6 2JA,T,N,F,NEWHAM,A,2018,51.539150,...,0,1,1.0,0.0,0.0,0.0,0.0,0.0,3,1.0
